In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from sklearn.model_selection import train_test_split
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

### Gradient Boosting Regression

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

# learning_rate는 기본이 0.1
# n_estimators = 100, max_depth = 3 : 깊이가 3인 트리 100개
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   random_state=0)

gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train)

print("훈련 세트 정확도 : {:.3f}".format(gbrt.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(gbrt.score(X_test, y_test)))

훈련 세트 정확도 : 1.000
테스트 세트 정확도 : 0.965


#### - 과대적합이 나타난다는 것을 알 수 있다.
#### - 트리 최대 깊이를 줄여서 사전 가지치기를 수행하거나 학습률(learning_rate)을 낮춰서 과대적합을 피할 수 있다.

In [4]:
from sklearn.datasets import make_moons
from sklearn.model_selection import cross_val_score

X, y = make_moons(n_samples=100, noise=0.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

cvscores = cross_val_score(gbrt, X, y, cv=10)
print("테스트 세트 정확도 CV = 10 : {:.3f}".format(cvscores.mean()))

cvscores = cross_val_score(gbrt, X, y, cv=5)
print("테스트 세트 정확도 CV = 5 : {:.3f}".format(cvscores.mean()))

테스트 세트 정확도 CV = 10 : 0.890
테스트 세트 정확도 CV = 5 : 0.850


#### - cross_val_score 함수를 통해 교차검증을 수행한다.
#### - cv를 크게 하여 교차검증을 할수록 테스트 세트 정확도가 향상된다.

In [6]:
# max_depth = 1 : 과적합 피하고 테스트 데이터 성능 향상
gbrt = GradientBoostingClassifier(random_state=0, max_depth=1)
gbrt.fit(X_train, y_train)

print("훈련 세트 정확도 : {:.3f}".format(gbrt.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}\n".format(gbrt.score(X_test, y_test)))

cvscores = cross_val_score(gbrt, X, y, cv=10)
print("테스트 세트 정확도 CV = 10 : {:.3f}".format(cvscores.mean()))

cvscores = cross_val_score(gbrt, X, y, cv=5)
print("테스트 세트 정확도 CV = 5 : {:.3f}".format(cvscores.mean()))

훈련 세트 정확도 : 0.973
테스트 세트 정확도 : 0.840

테스트 세트 정확도 CV = 10 : 0.890
테스트 세트 정확도 CV = 5 : 0.870


#### - 트리 최대 깊이(max_depth)를 작게 했더니 테스트 세트 정확도가 향상했다.
#### - 과대적합이 해소된 것을 알 수 있다.

In [8]:
# learning_rate = 0.01 : 테스트 데이터 정확도 조금 감소
gbrt = GradientBoostingClassifier(random_state=0, learning_rate=0.01)
gbrt.fit(X_train, y_train)

print("훈련 세트 정확도 : {:.3f}".format(gbrt.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}\n".format(gbrt.score(X_test, y_test)))

cvscores = cross_val_score(gbrt, X, y, cv=10)
print("테스트 세트 정확도 CV = 10 : {:.3f}".format(cvscores.mean()))

cvscores = cross_val_score(gbrt, X, y, cv=5)
print("테스트 세트 정확도 CV = 5 : {:.3f}".format(cvscores.mean()))

훈련 세트 정확도 : 0.907
테스트 세트 정확도 : 0.880

테스트 세트 정확도 CV = 10 : 0.870
테스트 세트 정확도 CV = 5 : 0.860


#### - 학습률(learning_rate)을 줄이자 이전 트리의 오차 보정을 약하게 하여 테스트 정확도가 조금 떨어진 것을 알 수 있다.
#### - 과대적합이 해소됐으나 정확도를 위해 적절한 수를 찾아서 조정해야 한다.

### SVM

In [10]:
from sklearn.svm import SVC

# 유방암 데이터
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   random_state=0)

svc = SVC()
svc.fit(X_train, y_train)

print("훈련 세트 정확도 : {:.2f}".format(svc.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.2f}".format(svc.score(X_test, y_test)))

훈련 세트 정확도 : 0.90
테스트 세트 정확도 : 0.94


In [11]:
# 성능 향상을 위한 데이터 전처리

# 훈련 세트 특성 별 최솟값 계산
min_on_training = X_train.min(axis=0)
# 훈련 세트 특성 별 (최댓값 - 최솟값) 범위 계산
range_on_training = (X_train - min_on_training).max(axis=0)

# 훈련 데이터에서 최솟값을 빼고 범위로 나누면
# 각 특성에 대한 최솟값 = 0, 최댓값 = 1
X_train_scaled = (X_train - min_on_training) / range_on_training

# 테스트 세트에도 같은 작업
min_on_test = X_test.min(axis=0)
range_on_test = (X_test - min_on_test).max(axis=0)
X_test_scaled = (X_test - min_on_test) / range_on_test

svc = SVC()
svc.fit(X_train_scaled, y_train)

print("훈련 세트 정확도 : {:.3f}".format(svc.score(X_train_scaled, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(svc.score(X_test_scaled, y_test)))

훈련 세트 정확도 : 0.984
테스트 세트 정확도 : 0.909


#### - 성능 향상을 위해 데이터 전처리를 수행했는데 오히려 정확도가 떨어졌다.

In [12]:
svc = SVC(C=1000)
svc.fit(X_train_scaled, y_train)

print("훈련 세트 정확도 : {:.3f}".format(svc.score(X_train_scaled, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(svc.score(X_test_scaled, y_test)))

훈련 세트 정확도 : 1.000
테스트 세트 정확도 : 0.797


#### - 규제매개변수 c를 크게 하면 제약이 약해져서 과대적합이 발생한다.

In [16]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
lsvc.fit(X_train, y_train)

print("훈련 세트 정확도 : {:.2f}".format(lsvc.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.2f}".format(lsvc.score(X_test, y_test)))

훈련 세트 정확도 : 0.92
테스트 세트 정확도 : 0.94


C:\Users\kym28\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


#### - LinearSVC는 SVC보다 선형 분류 가능한 데이터에 적합한 모델이다.
#### - 테스트 세트에서의 차이는 없으나 훈련 세트 정확도는 LinearSVC가 더 높게 나온 것을 확인할 수 있다.

### 신경망 MLP

In [14]:
from sklearn.neural_network import MLPClassifier

# Cancer 데이터 Z-Score 표준화
# 훈련 세트 각 특성의 평균을 계산
mean_on_train = X_train.mean(axis=0)
# 훈련 세트 각 특성의 표준 편차 계산
std_on_train = X_train.std(axis=0)

# 데이터에서 평균을 빼고 표준 편차로 나누면
# 평균 0, 표준 편차 1인 데이터로 변환
X_train_scaled = (X_train - mean_on_train) / std_on_train
X_test_scaled = (X_test - mean_on_train) / std_on_train

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train_scaled, y_train)

print("훈련 세트 정확도 : {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(mlp.score(X_test_scaled, y_test)))

훈련 세트 정확도 : 0.991
테스트 세트 정확도 : 0.965


C:\Users\kym28\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
# 반복횟수를 증가 (default = 200, SVC accuracy : 0.972)
mlp = MLPClassifier(max_iter=1000, random_state=0)

mlp.fit(X_train_scaled, y_train)

print("훈련 세트 정확도 : {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(mlp.score(X_test_scaled, y_test)))

훈련 세트 정확도 : 1.000
테스트 세트 정확도 : 0.972


In [17]:
# alpha 증가
mlp = MLPClassifier(max_iter=1000, alpha=1, random_state=0)

mlp.fit(X_train_scaled, y_train)

print("훈련 세트 정확도 : {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(mlp.score(X_test_scaled, y_test)))

훈련 세트 정확도 : 0.988
테스트 세트 정확도 : 0.972


#### - alpha를 증가시켜서 규제를 강화해도 반복 횟수 증가로 인해 성능이 향상됐기 때문에 큰 변화가 없다.